# DC Medium Office Metamodel

In [7]:
# import libraries

import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
from plotly.subplots import make_subplots

output_dir = os.path.join("output", "dc_data")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def clean_up_name(name):
    return name.replace('/', '_').replace(' ', '_')

In [8]:
df = pd.read_excel('benchmark/Building_Energy_Benchmarks.xlsx')
reporting_years = [yr for yr in df['REPORTINGYEAR'].unique() if yr >= 2016]
reporting_years

# downselect the larger dataframe to only the reporting years of interest
df = df[df['REPORTINGYEAR'].isin(reporting_years)]

In [9]:
building_types = df['PRIMARYPROPERTYTYPE_SELFSELECT'].unique()
print(f"There are {len(building_types)} building types")

vars = ['PRIMARYPROPERTYTYPE_SELFSELECT', 'REPORTINGYEAR', 'SITEEUI_KBTU_FT']
group_by = ['PRIMARYPROPERTYTYPE_SELFSELECT', 'REPORTINGYEAR' ]
rename = {"SITEEUI_KBTU_FT": "Count"}
sort_by = ['PRIMARYPROPERTYTYPE_SELFSELECT']
data_to_plot = df[vars].groupby(group_by).count().reset_index().rename(columns=rename)
# only show where count > n
data_to_plot = data_to_plot[data_to_plot['Count'] > 15].sort_values(by=sort_by)
data_to_plot['REPORTINGYEAR'] = data_to_plot['REPORTINGYEAR'].apply(str)

# Save off the mostly reported building
common_building_types = data_to_plot['PRIMARYPROPERTYTYPE_SELFSELECT'].unique()

fig = px.bar(data_to_plot, x="Count",
             y='PRIMARYPROPERTYTYPE_SELFSELECT',
             color='REPORTINGYEAR',
             orientation='h',
)
fig.update_layout(
    title='Property Types in DC (>15)',
    yaxis=None,
)
fig.show()
fig.write_image(f"{output_dir}/building_types.png")

There are 54 building types


In [10]:
variables_to_plot = [
    {
        'column_name': 'YEARBUILT',
        'clean_name': 'year_built',
        'display_name': 'Year Built',
        'units': 'Year',
    },
    {
        'column_name': 'SITEEUI_KBTU_FT',
        'clean_name': 'site_eui',
        'display_name': 'Site EUI',
        'units': 'kBTU/ft2/year',
    },
    {
        'column_name': 'ELECTRICITYUSE_GRID_KWH',
        'clean_name': 'electricity_use',
        'display_name': 'Electricity Use',
        'units': 'kWh/year',
    },
    {
        'column_name': 'NATURALGASUSE_THERMS',
        'clean_name': 'gas_use',
        'display_name': 'Natural Gas Use',
        'units': 'Therms/year',
    },
    {
        'column_name': 'REPORTEDBUILDINGGROSSFLOORAREA',
        'clean_name': 'floor_area',
        'display_name': 'Gross Floor Area',
        'units': 'sqft',
    },
]

# for report_year in reporting_years:
for var in variables_to_plot:
    for building_type in common_building_types:
        # fig = make_subplots(rows=3, cols=1)
        fig = go.Figure()
        
        for index, reporting_year in enumerate(reporting_years):
            trace = go.Histogram(
                x=df[(df['PRIMARYPROPERTYTYPE_SELFSELECT'] == building_type) & (df['REPORTINGYEAR'] == reporting_year)][var['column_name']],
                name=f"{reporting_year}",
                histnorm='probability',
            )
            # fig.append_trace(trace, index+1, 1 )
            fig.add_trace(trace)
    
        fig.update_layout(
            barmode='overlay',
            title_text=f"{building_type} {var['display_name']} Distribution", 
            xaxis_title_text=f"{var['display_name']} ({var['units']})",
            yaxis_title_text='Probability',
        )
        fig.update_traces(opacity=0.50)
    
    #     fig.show()
        fig.write_image(f"{output_dir}/{clean_up_name(building_type)}_{var['clean_name']}.png")

    

In [34]:
# sample 3 office buildings at random
# eui between 50 and 75
# year built between 1980 and 2010
building_type = (df['PRIMARYPROPERTYTYPE_SELFSELECT'] == 'Office')
eui_range = (df['SITEEUI_KBTU_FT'] >= 30) & (df['SITEEUI_KBTU_FT'] < 90)
year_range = (df['YEARBUILT'] >= 1980) & (df['YEARBUILT'] <= 2010)
floor_area_range = (df['REPORTEDBUILDINGGROSSFLOORAREA'] >= 20000) & (df['REPORTEDBUILDINGGROSSFLOORAREA'] <= 75000)
reporting_range = df['REPORTINGYEAR'] == 2018

subset_df = df[eui_range & year_range & building_type & floor_area_range & reporting_range]
print(subset_df.describe())
print(subset_df['NATURALGASUSE_THERMS'].describe())



               X          Y      OBJECTID  PMPROPERTYID  REPORTINGYEAR  \
count  13.000000  13.000000     13.000000  1.300000e+01           13.0   
mean  -77.034808  38.902101   6831.153846  4.229315e+06         2018.0   
std     0.017463   0.007110   3456.336400  1.492296e+06            0.0   
min   -77.066534  38.884149    270.000000  1.865876e+06         2018.0   
25%   -77.042468  38.900520   5263.000000  3.858497e+06         2018.0   
50%   -77.037229  38.903406   5775.000000  3.994644e+06         2018.0   
75%   -77.030934  38.905129   7941.000000  4.561384e+06         2018.0   
max   -76.991691  38.916092  13004.000000  6.723153e+06         2018.0   

            WARD    YEARBUILT  TAXRECORDFLOORAREA  \
count  13.000000    13.000000           13.000000   
mean    2.538462  1989.769231        80796.153846   
std     1.330124     8.217399        29597.452767   
min     2.000000  1981.000000        55474.000000   
25%     2.000000  1985.000000        61506.000000   
50%     2.00000

In [38]:
sample_df = subset_df.sample(3, random_state=4815)

# save to CSV for use in another script
sample_df.to_csv('buildings_to_optimize.csv')

In [38]:

# for year in years:
#     x_data = [x for x in range(24)]
    
#     # create some plots
#     # fig = make_subplots(rows=3, cols=1)
#     fig = make_subplots(specs=[[{'secondary_y': True}]])
#     fig.add_trace(go.Scatter(x=x_data, y=load_day, name="Building Load (kW)"),)
#     fig.add_trace(go.Scatter(x=x_data, y=actual_p_batt, name="Battery Charge/Discharge (kW)"),)
#     fig.add_trace(go.Scatter(x=x_data, y=actual_p_g, name="Power Delivered from Grid (kW)"),)
#     fig.add_trace(go.Scatter(x=x_data, y=actual_e_batt, name="Battery Charge (kWh)"),  secondary_y=True)
#     fig.update_layout(title_text=f'MPC (Horizon = {horizon} Hours), Total Cost = ${total_cost:.2f}', yaxis_range=[-2.2,15])
#     fig.update_xaxes(title_text='Time (Hour of Day)')
#     fig.update_yaxes(title_text='Power (kW)')
#     fig.update_yaxes(title_text='Battery Charge (kWh)', secondary_y=True, range=[-2.2,15])
#     fig.show()
# print(df['REPORTINGYEAR'].describe())
# df.columns

0        80.6
1        91.0
2        68.6
3        57.9
4       111.1
5        43.5
6        77.9
7        88.4
8        46.8
9        93.2
10       66.7
11      120.9
12      132.0
13       67.6
14       69.3
15       98.6
16       44.8
17        NaN
18       85.9
19       71.8
20       56.8
21       49.1
22      117.8
23       44.2
24       88.4
25       39.0
26       94.8
27       46.0
28      172.3
29       18.9
        ...  
9334     84.5
9335     51.3
9336    104.2
9337     45.8
9338    148.2
9339     91.4
9340    100.7
9341     46.2
9342     63.7
9343    299.9
9344     63.2
9345     92.1
9346     62.4
9347    102.9
9348    120.5
9349     58.5
9350     70.8
9351     44.6
9352    299.6
9353     91.3
9354     52.7
9355     77.6
9356     41.9
9357    101.6
9358     30.1
9359     32.4
9360     56.3
9361     95.2
9362    140.1
9363    163.7
Name: SITEEUI_KBTU_FT, Length: 1798, dtype: float64

In [50]:
# Test running a weather file

analysis = AnalysisDefinition('analysis_definitions/medium_office.json')
analysis.load_weather_file('../bem/weather/USA_VA_Arlington/USA_VA_Arlington-Ronald.Reagan.Washington.Natl.AP.724050_TMY3.epw')

# convert the analysis definition to a dataframe for use in the metamodel
data = analysis.as_dataframe()
data = metamodel.yhats(data, 'RF', ['HeatingElectricity', 'CoolingElectricity'])
data['RF_CoolingElectricity'] = data['RF_CoolingElectricity'] / (3600 * 1000)  # J to kWh
data['RF_HeatingElectricity'] = data['RF_HeatingElectricity'] / (3600 * 1000)  # J to kWh

# describe the data
print(data.describe())

# view a couple single rows
print(data.iloc[0])
print(data.iloc[3000])

# Create heat maps of the modeled data
output_dir = 'analysis_output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

responses = [
    'RF_HeatingElectricity',
    'RF_CoolingElectricity',
]
for response in responses:
    heatdata = data[["DayOfYear", "Hour", response]].pivot("DayOfYear", "Hour", response)
    f, ax = plt.subplots(figsize=(5, 12))
    sns.heatmap(heatdata)
    filename = f"{output_dir}/{response.replace(' ', '_')}.png"
    plt.savefig(filename)
    plt.close('all')


Parsing Location
Parsing Design Conditions
Parsing Typical / Extreme Periods
Parsing Ground Temperatures
Parsing Holidays / Daylight Savings
Parsing Comments 1
Parsing Comments 2
Parsing Data Periods
              year        Month    DayofWeek         Hour  minute  \
count  8760.000000  8760.000000  8760.000000  8760.000000  8760.0   
mean   1998.915068     6.526027    15.720548    12.500000     0.0   
std       3.189362     3.448048     8.796749     6.922582     0.0   
min    1994.000000     1.000000     1.000000     1.000000     0.0   
25%    1996.000000     4.000000     8.000000     6.750000     0.0   
50%    1999.000000     7.000000    16.000000    12.500000     0.0   
75%    2001.000000    10.000000    23.000000    18.250000     0.0   
max    2005.000000    12.000000    31.000000    24.000000     0.0   

       SiteOutdoorAirDrybulbTemperature    dew_point  \
count                       8760.000000  8760.000000   
mean                          14.127454     7.403836   
std       

In [62]:
import plotly.express as px

fig = make_subplots() #(specs=[[{'secondary_y': True}]])
# fig.add_trace(go.Scatter(x=data['datetime'], y=[''], name="Heating Electricty (kW)"),)

fig = px.line(data, x='datetime', y='RF_HeatingElectricity', title='Modeled Heating Electricity')
fig.show()

fig = px.line(data, x='datetime', y='RF_CoolingElectricity', title='Modeled Cooling Electricity')
fig.show()


In [ ]:
for horizon in horizons:
    actual_cost, actual_e_batt, actual_p_batt, actual_p_g, optimals = run_mpc(horizon)
    # print(f"Actual Cost: {actual_cost}")
    # print(f"Battery Charge/Discharge {actual_p_batt}")
    # print(f"Battery Charge {actual_e_batt}")
    # print(f"Building Load {load_day}")    
    # print(f"Power from Grid {actual_p_g}")    
    total_cost = sum(actual_cost)
    x_data = [x for x in range(24)]
    
    # create some plots
    # fig = make_subplots(rows=3, cols=1)
    fig = make_subplots(specs=[[{'secondary_y': True}]])
    fig.add_trace(go.Scatter(x=x_data, y=load_day, name="Building Load (kW)"),)
    fig.add_trace(go.Scatter(x=x_data, y=actual_p_batt, name="Battery Charge/Discharge (kW)"),)
    fig.add_trace(go.Scatter(x=x_data, y=actual_p_g, name="Power Delivered from Grid (kW)"),)
    fig.add_trace(go.Scatter(x=x_data, y=actual_e_batt, name="Battery Charge (kWh)"),  secondary_y=True)
    fig.update_layout(title_text=f'MPC (Horizon = {horizon} Hours), Total Cost = ${total_cost:.2f}', yaxis_range=[-2.2,15])
    fig.update_xaxes(title_text='Time (Hour of Day)')
    fig.update_yaxes(title_text='Power (kW)')
    fig.update_yaxes(title_text='Battery Charge (kWh)', secondary_y=True, range=[-2.2,15])
    fig.show()